In [1]:
import plaidml.keras
plaidml.keras.install_backend()
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

In [2]:
# Importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers

import time 

In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
df = pd.read_csv('../data/num_data.csv')

# import io
# df = pd.read_csv(io.BytesIO(uploaded['num_data.csv']))


### Data Processing

In [5]:
df = pd.read_csv('../data/num_data.csv')

In [6]:
POLLUTION = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']

In [7]:
WEATHER = ['PM2.5', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM']

In [8]:
dataset = df

In [9]:
dataset.shape

(420768, 16)

In [11]:
data_size = dataset.shape[0]
train_size=int(data_size * 0.6)
test_size = int(data_size * 0.2)
valid_size = data_size - train_size - test_size

In [12]:
training_set = dataset[:train_size].iloc[:,4:16].values
valid_set = dataset[train_size:train_size+valid_size].iloc[:,4:16].values
test_set = dataset[data_size-test_size:].iloc[:,4:16].values

In [13]:
y = dataset.iloc[:,0].values
y = y.reshape(-1,1)
n_feature = training_set.shape[1]
y.shape

(420768, 1)

In [14]:
# Scaling the dataset
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)
valid_set_scaled = sc.fit_transform(valid_set)
test_set_scaled = sc.fit_transform(test_set)

sc_y = MinMaxScaler(feature_range=(0,1))
y_scaled = sc_y.fit_transform(y)

In [15]:
# convert dataset into sequences, where n_steps_in is the input sequence lengith, 
# and n_steps_out is the output sequence length 
def convert_to_sequences(sequences, n_steps_in, n_steps_out):
    X_, y_ = [], []
    for i in range(len(sequences)):
        tail_x = i + n_steps_in
        out_tail_x = tail_x + n_steps_out-1
        if out_tail_x > len(sequences):
            break
        seq_x, seq_y = sequences[i:tail_x, :], sequences[tail_x-1:out_tail_x, 0]
        X_.append(seq_x)
        y_.append(seq_y)
    return np.array(X_), np.array(y_)

In [16]:
n_steps_in = 12
n_steps_out = 12
X_train, y_train = convert_to_sequences(training_set_scaled, n_steps_in, n_steps_out)
X_valid, y_valid = convert_to_sequences(valid_set_scaled, n_steps_in, n_steps_out)
X_test, y_test = convert_to_sequences(test_set_scaled, n_steps_in, n_steps_out)

## Grid Search Control 


In [17]:
n_activation = ['tanh', 'sigmoid', 'relu']
act = n_activation[0]

n_learn_rate = [0.01, 0.1, 0.2]
lr = n_learn_rate[0]

n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
opt = n_optimizers[0]

n_epoches = [50]
epoch = n_epoches[0]

n_batch_size = [32, 256, 1024]
batch = n_batch_size[-1]

n_of_neurons = [10, 50, 200]
neuron = n_of_neurons[1]

INFO:plaidml:Opening device "llvm_cpu.0"


In [18]:
rmse_df = pd.DataFrame(columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'epoch', 
                               'batch', 'neuron'])

In [19]:
for act in n_activation:
    for lr in n_learn_rate:
        n_optimizers = [optimizers.Adam(lr=lr), optimizers.RMSprop(lr=lr), optimizers.SGD(lr=lr)]
        for opt in n_optimizers:          
            CBGRU = Sequential()
            CBGRU.add(Conv1D(filters=64, kernel_size=6, activation='tanh', input_shape=(X_train.shape[1],n_feature)))
            CBGRU.add(MaxPooling1D(pool_size=4))
            CBGRU.add(Dropout(0.2))  
            CBGRU.add(Bidirectional(GRU(units=neuron, return_sequences=False, input_shape=(X_train.shape[1],n_feature), activation=act)))
            CBGRU.add(Dense(units=n_steps_out))
            CBGRU.compile(optimizer=opt,loss='mean_squared_error')

            
            regressor = CBGRU
            model = 'CBGRU'
    
            print('training start for', model)    
            start = time.process_time()
            regressor.fit(X_train,y_train,epochs=epoch,batch_size=batch)
            train_time = round(time.process_time() - start, 2)

            print('results for training set')
            y_train_pred = regressor.predict(X_train)
            train_rmse = return_rmse(y_train,y_train_pred)

            print('results for valid set')
            y_valid_pred = regressor.predict(X_valid)
            valid_rmse = return_rmse(y_valid,y_valid_pred)

            print('results for test set')
            y_test_pred = regressor.predict(X_test)
            test_rmse = return_rmse(y_test,y_test_pred)
    
    
            one_df = pd.DataFrame([[model, train_rmse, valid_rmse, test_rmse, train_time, epoch, batch, neuron]],
                          columns=['Model', 'train_rmse', 'valid_rmse', 'test_rmse', 'train_time', 'epoch', 
                               'batch', 'neuron'])
            rmse_df = pd.concat([rmse_df, one_df])

# save the rmse results 
rmse_df.to_csv('../cbgru_grid_search_v2.csv')

            

training start for CBGRU
Epoch 1/50
252438/252438 [==============================] - 36s 142us/step - loss: 0.0041
Epoch 2/50
252438/252438 [==============================] - 30s 121us/step - loss: 0.0034
Epoch 3/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0033
Epoch 4/50
252438/252438 [==============================] - 28s 111us/step - loss: 0.0033
Epoch 5/50
252438/252438 [==============================] - 28s 110us/step - loss: 0.0032
Epoch 6/50
252438/252438 [==============================] - 28s 110us/step - loss: 0.0032
Epoch 7/50
252438/252438 [==============================] - 28s 110us/step - loss: 0.0032
Epoch 8/50
252438/252438 [==============================] - 29s 113us/step - loss: 0.0032
Epoch 9/50
252438/252438 [==============================] - 29s 113us/step - loss: 0.0032
Epoch 10/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0031
Epoch 11/50
252438/252438 [==============================] - 28s 112us/ste

Epoch 40/50
252438/252438 [==============================] - 35s 140us/step - loss: 0.0033
Epoch 41/50
252438/252438 [==============================] - 36s 142us/step - loss: 0.0033
Epoch 42/50
252438/252438 [==============================] - 36s 143us/step - loss: 0.0033
Epoch 43/50
252438/252438 [==============================] - 36s 141us/step - loss: 0.0033
Epoch 44/50
252438/252438 [==============================] - 37s 145us/step - loss: 0.0033
Epoch 45/50
252438/252438 [==============================] - 36s 144us/step - loss: 0.0033
Epoch 46/50
252438/252438 [==============================] - 37s 146us/step - loss: 0.0032
Epoch 47/50
252438/252438 [==============================] - 36s 142us/step - loss: 0.0032
Epoch 48/50
252438/252438 [==============================] - 36s 143us/step - loss: 0.0032
Epoch 49/50
252438/252438 [==============================] - 37s 148us/step - loss: 0.0032
Epoch 50/50
252438/252438 [==============================] - 39s 155us/step - loss: 0.0032

252438/252438 [==============================] - 31s 122us/step - loss: 0.0044
Epoch 17/50
252438/252438 [==============================] - 30s 118us/step - loss: 0.0043
Epoch 18/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0043
Epoch 19/50
252438/252438 [==============================] - 21s 82us/step - loss: 0.0043
Epoch 20/50
252438/252438 [==============================] - 24s 94us/step - loss: 0.0042
Epoch 21/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0042
Epoch 22/50
252438/252438 [==============================] - 30s 120us/step - loss: 0.0042
Epoch 23/50
252438/252438 [==============================] - 31s 123us/step - loss: 0.0042
Epoch 24/50
252438/252438 [==============================] - 31s 124us/step - loss: 0.0042
Epoch 25/50
252438/252438 [==============================] - 31s 122us/step - loss: 0.0041
Epoch 26/50
252438/252438 [==============================] - 30s 119us/step - loss: 0.0041
Epoch 27/50
2

Epoch 4/50
252438/252438 [==============================] - 33s 131us/step - loss: 24.0158
Epoch 5/50
252438/252438 [==============================] - 34s 134us/step - loss: 24.0379
Epoch 6/50
252438/252438 [==============================] - 34s 133us/step - loss: 24.0221
Epoch 7/50
252438/252438 [==============================] - 33s 132us/step - loss: 24.0143
Epoch 8/50
252438/252438 [==============================] - 34s 135us/step - loss: 24.0167
Epoch 9/50
252438/252438 [==============================] - 34s 133us/step - loss: 24.0139
Epoch 10/50
252438/252438 [==============================] - 33s 129us/step - loss: 24.0171
Epoch 11/50
252438/252438 [==============================] - 35s 138us/step - loss: 24.0168
Epoch 12/50
252438/252438 [==============================] - 38s 151us/step - loss: 24.0156
Epoch 13/50
252438/252438 [==============================] - 37s 146us/step - loss: 24.0162
Epoch 14/50
252438/252438 [==============================] - 38s 152us/step - loss: 24

Epoch 30/50
252438/252438 [==============================] - 40s 157us/step - loss: 0.0033
Epoch 31/50
252438/252438 [==============================] - 40s 157us/step - loss: 0.0032
Epoch 32/50
252438/252438 [==============================] - 42s 168us/step - loss: 0.0032
Epoch 33/50
252438/252438 [==============================] - 39s 153us/step - loss: 0.0032
Epoch 34/50
252438/252438 [==============================] - 42s 168us/step - loss: 0.0032
Epoch 35/50
252438/252438 [==============================] - 34s 136us/step - loss: 0.0032
Epoch 36/50
252438/252438 [==============================] - 27s 106us/step - loss: 0.0032
Epoch 37/50
252438/252438 [==============================] - 32s 127us/step - loss: 0.0032
Epoch 38/50
252438/252438 [==============================] - 37s 148us/step - loss: 0.0032
Epoch 39/50
252438/252438 [==============================] - 42s 168us/step - loss: 0.0032
Epoch 40/50
252438/252438 [==============================] - 43s 171us/step - loss: 0.0032

252438/252438 [==============================] - 28s 112us/step - loss: 0.0059
Epoch 7/50
252438/252438 [==============================] - 30s 117us/step - loss: 0.0057
Epoch 8/50
252438/252438 [==============================] - 29s 113us/step - loss: 0.0055
Epoch 9/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0054
Epoch 10/50
252438/252438 [==============================] - 28s 110us/step - loss: 0.0052
Epoch 11/50
252438/252438 [==============================] - 28s 113us/step - loss: 0.0051
Epoch 12/50
252438/252438 [==============================] - 29s 113us/step - loss: 0.0049
Epoch 13/50
252438/252438 [==============================] - 30s 119us/step - loss: 0.0048
Epoch 14/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0047
Epoch 15/50
252438/252438 [==============================] - 29s 115us/step - loss: 0.0047
Epoch 16/50
252438/252438 [==============================] - 28s 110us/step - loss: 0.0046
Epoch 17/50
25

252438/252438 [==============================] - 19s 75us/step - loss: 0.0041
Epoch 34/50
252438/252438 [==============================] - 23s 93us/step - loss: 0.0040
Epoch 35/50
252438/252438 [==============================] - 28s 109us/step - loss: 0.0040
Epoch 36/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0040
Epoch 37/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0040
Epoch 38/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0040
Epoch 39/50
252438/252438 [==============================] - 29s 115us/step - loss: 0.0040
Epoch 40/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0040
Epoch 41/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0040
Epoch 42/50
252438/252438 [==============================] - 30s 117us/step - loss: 0.0040
Epoch 43/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0040
Epoch 44/50
2

252438/252438 [==============================] - 30s 117us/step - loss: 0.0036
Epoch 22/50
252438/252438 [==============================] - 28s 111us/step - loss: 0.0036
Epoch 23/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0036
Epoch 24/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0036
Epoch 25/50
252438/252438 [==============================] - 28s 113us/step - loss: 0.0036
Epoch 26/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0036
Epoch 27/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0036
Epoch 28/50
252438/252438 [==============================] - 29s 116us/step - loss: 0.0036
Epoch 29/50
252438/252438 [==============================] - 28s 112us/step - loss: 0.0036
Epoch 30/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0035
Epoch 31/50
252438/252438 [==============================] - 29s 114us/step - loss: 0.0036
Epoch 32/50

252438/252438 [==============================] - 17s 66us/step - loss: 0.0093
Epoch 50/50
252438/252438 [==============================] - 17s 68us/step - loss: 0.0094
results for training set
results for valid set
results for test set
training start for CBGRU
Epoch 1/50
252438/252438 [==============================] - 20s 80us/step - loss: 0.0076
Epoch 2/50
252438/252438 [==============================] - 16s 65us/step - loss: 0.0063
Epoch 3/50
252438/252438 [==============================] - 17s 68us/step - loss: 0.0057
Epoch 4/50
252438/252438 [==============================] - 17s 66us/step - loss: 0.0053
Epoch 5/50
252438/252438 [==============================] - 17s 69us/step - loss: 0.0051
Epoch 6/50
252438/252438 [==============================] - 17s 66us/step - loss: 0.0049
Epoch 7/50
252438/252438 [==============================] - 17s 66us/step - loss: 0.0048
Epoch 8/50
252438/252438 [==============================] - 17s 66us/step - loss: 0.0047
Epoch 9/50
252438/252438 [=

252438/252438 [==============================] - 17s 66us/step - loss: 0.0039
Epoch 28/50
252438/252438 [==============================] - 17s 65us/step - loss: 0.0038
Epoch 29/50
252438/252438 [==============================] - 17s 68us/step - loss: 0.0038
Epoch 30/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0038
Epoch 31/50
252438/252438 [==============================] - 17s 66us/step - loss: 0.0038
Epoch 32/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0038
Epoch 33/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0038
Epoch 34/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0038
Epoch 35/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0038
Epoch 36/50
252438/252438 [==============================] - 17s 68us/step - loss: 0.0038
Epoch 37/50
252438/252438 [==============================] - 17s 67us/step - loss: 0.0037
Epoch 38/50
252438/252